<a href="https://colab.research.google.com/github/Alib8866/-Classifying-the-Mode-of-a-Sounds/blob/main/MVP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import matplotlib.pyplot as plt

df_train = pd.read_csv("/kaggle/input/network-intrusion-detection/Train_data.csv")

print(df_train.info())

In [ ]:
data_train = pd.get_dummies(df_train,columns = ["protocol_type", "flag", "service", "class"], drop_first=True)

data_train.head()

In [ ]:
y_data = data_train.class_normal.values
x_data = data_train.drop(["class_normal"], axis=1)
y_data = y_data.astype('int64') #we convert dtype of y_data to int64 here for a positive decrease in cost values.

print("Y datas : ", y_data) #if it is normal 1 else 0.
print("X datas : \n", x_data.head())

In [ ]:
x = (x_data-np.min(x_data))/(np.max(x_data)-np.min(x_data)).values

x.head(n=10

In [ ]:
x[['num_outbound_cmds','is_host_login']].head()

In [ ]:
x.drop(["num_outbound_cmds", "is_host_login"], axis = 1, inplace=True)
x.head()

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y_data, test_size=0.2, random_state=42)

x_train = x_train.T
x_test = x_test.T
y_train = y_train.T
y_test = y_test.T

print("x_train :", x_train.shape)
print("x_test :", x_test.shape)
print("y_train :", y_train.shape)
print("y_test :", y_test.shape)

In [ ]:
def initialize_weigths_and_bias(dimension):
    w = np.full((dimension,1),0.01)
    b = 0.0
    return w,b

def sigmoid(z):
    y_head = 1/(1 + np.exp(-z))
    return y_head

In [ ]:
def forward_backward_propagation(w,b,x_train,y_train):
    #forward propagation
    z = np.dot(w.T, x_train) + b
    y_head = sigmoid(z)
    loss = -y_train*np.log(y_head) - (1-y_train)*np.log(1-y_head)
    cost = (np.sum(loss)) / x_train.shape[1]
    
    #backward propagation 
    derivative_weight = (np.dot(x_train,((y_head-y_train).T))) / x_train.shape[1]    
    derivative_bias = np.sum(y_head-y_train) / x_train.shape[1]    
    gradients = {"derivative_weight" : derivative_weight, "derivative_bias" : derivative_bias}
    return cost, gradients

In [ ]:
def update(w,b,x_train,y_train,learning_rate,number_of_iteration):
    cost_list = []
    cost_list2 = []
    index = []
    
    #updating(learning) parametersis number_of_iteration times 
    for i in range(number_of_iteration):
        #make forward and backwar propagation to find cost and gradients
        cost, gradients = forward_backward_propagation(w, b, x_train, y_train)
        cost_list.append(cost)
        #lets update 
        w = w - learning_rate * gradients["derivative_weight"]
        b = b - learning_rate * gradients["derivative_bias"]
        if i % 3 == 0 :
            cost_list2.append(cost)
            index.append(i)
            print("cost after iteration %i : %f" %(i,cost))
            
    #store updated parameters in a parameters dictionary
    parameters = {"weight":w,"bias":b}
    plt.plot(index,cost_list2)
    plt.xticks(index,rotation='vertical')
    plt.xlabel("Number of iteration")
    plt.ylabel("Cost")
    plt.show()
    return parameters, gradients, cost_list

In [ ]:
def predict(w,b,x_test):  
    z = sigmoid(np.dot(w.T,x_test)+b) 
    Y_prediction = np.zeros((1,x_test.shape[1])) # the length of Y-prediction should be as the number of samples.
    # if y_head is less than 0.5 the class is 0, else 1.
    for i in range(z.shape[1]):
        if z[0,i] <= 0.5:
            Y_prediction[0,i] = 0
        else:
            Y_prediction[0,i] = 1
        
    return Y_prediction

In [ ]:
def logistic_regression(x_train,y_train,x_test,y_test,learning_rate,num_iterations):
    dimension = x_train.shape[0]
    w,b = initialize_weigths_and_bias(dimension)
    
    #run learning algorithm
    parameters, gradients, cost_list = update(w,b,x_train,y_train,learning_rate,num_iterations)
    
    #We get prediction values with the test data and the trained parameters we give.
    y_prediction_test = predict(parameters["weight"],parameters["bias"], x_test) 
    
    error_value = np.mean(np.abs(y_prediction_test - y_test)) * 100    
    print("test accuracy: {} %".format(100 - error_value ))
    
    return  y_prediction_test, y_test

In [ ]:
y_prediction_test, y_test = logistic_regression(x_train, y_train, x_test, y_test, learning_rate=3, num_iterations=100)

In [ ]:
y_test_values = np.zeros((1,x_test.shape[1]))
y_test_values[:1,:10] = y_test[:10] 

print("real : ", y_test_values[:1,:10])
print("pred : ", y_prediction_test[:1,:10])

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=200)
lr.fit(x_train.T,y_train.T)
print("test acuracy : {}".format(lr.score(x_test.T,y_test.T)))